# Q2 — Counterfactual Mechanism Comparison (主线)

目的：比较不同投票机制（Percent vs Rank，以及 Judge Save 变体）在历史每周数据下，会导致多大程度的淘汰差异。

对应主线代码：
- `src/mcm2026/pipelines/mcm2026c_q2_counterfactual_simulation.py`
- `src/mcm2026/visualizations/q2_visualizations.py`

主要输出：
- `outputs/tables/mcm2026c_q2_mechanism_comparison.csv`

说明：Q2 依赖 Q1 的 posterior summary（因为需要 fan_share / fan_rank）。

## 先用一句话说清楚 Q2（口语版）

Q1 给了我们“每周观众大概怎么投”的估计。
Q2 做的事情是：
> **把同一份“观众投票”放进不同规则里，看淘汰结果会不会变。**

### 你可以把 Q2 理解成什么？
理解成“历史回放 + 换规则重放”：
- 现实中 DWTS 用一种规则
- 我们把规则换成另一种（percent / rank / judge save），
- 看哪些周会出现不同淘汰、哪些季度整体差异更大。

### Q2 输出表里最关键的几个字段（论文口径）
- `diff_weeks_percent_vs_rank`：同一季里，有多少周 percent vs rank 的淘汰不同（越大说明规则差异越明显）
- `match_rate_*`：该规则重放的淘汰是否与真实历史一致（越高说明越“贴近历史”）

这张表通常用于写“机制差异不只是理论，实际会改变哪些季度的淘汰路径”。

In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

import pandas as pd

def _find_repo_root(start: Path | None = None) -> Path:
    here = Path.cwd() if start is None else Path(start)
    for p in [here, *here.parents]:
        if (p / 'pyproject.toml').exists():
            return p
    return here

REPO_ROOT = _find_repo_root()
SRC_DIR = REPO_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

REPO_ROOT


In [ ]:
# 工具函数：在 notebook 里自动展示 outputs/figures 下的图片（优先 TIFF，其次提示 EPS 路径）
from pathlib import Path


def show_saved_figures(question: str, stems: list[str]) -> None:
    """Display figures saved by visualization scripts.

    - Prefer TIFF (good for notebook preview)
    - EPS usually cannot be previewed in Jupyter without额外系统依赖，所以这里只打印路径
    """

    try:
        from IPython.display import Image, display
    except Exception:
        print('⚠️ IPython not available; cannot display images inline.')
        return

    q = question.strip().lower()
    tiff_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'tiff'
    eps_dir = Path(REPO_ROOT) / 'outputs' / 'figures' / q / 'eps'

    shown = 0
    for stem in stems:
        tiff_fp = tiff_dir / f'{stem}.tiff'
        eps_fp = eps_dir / f'{stem}.eps'

        if tiff_fp.exists():
            display(Image(filename=str(tiff_fp)))
            shown += 1
        elif eps_fp.exists():
            print('EPS figure exists (not previewed inline):', eps_fp)

    if shown == 0:
        print('⚠️ No TIFF figures found under:', tiff_dir)


# 用法示例（Q2）：
# show_saved_figures('q2', ['q2_judge_save_impact'])

## 一键运行（推荐给新队友/论文手）

运行下面这一格会自动补齐依赖并完成：
- Q0 → Q1 → Q2 的主线计算
- 生成 Q2 的 TIFF/EPS 图
- 在 notebook 里直接展示关键 TIFF 图

如果你只是想看“结论+图”，跑这一格就够了。

In [ ]:
import traceback

try:
    import sys
    from pathlib import Path

    import pandas as pd

    # ---- Make this cell self-contained (no dependency on earlier cells) ----
    def _find_repo_root(start: Path | None = None) -> Path:
        here = Path.cwd() if start is None else Path(start)
        for p in [here, *here.parents]:
            if (p / 'pyproject.toml').exists():
                return p
        return here

    if 'REPO_ROOT' not in globals() or not isinstance(globals().get('REPO_ROOT'), Path):
        REPO_ROOT = _find_repo_root()

    SRC_DIR = REPO_ROOT / 'src'
    if str(SRC_DIR) not in sys.path:
        sys.path.insert(0, str(SRC_DIR))

    try:
        from IPython.display import display
    except Exception:
        def display(x):
            return x

    from mcm2026.pipelines import (
        mcm2026c_q0_build_weekly_panel,
        mcm2026c_q1_smc_fan_vote,
        mcm2026c_q2_counterfactual_simulation,
    )
    from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
    from mcm2026.visualizations.q2_visualizations import generate_all_q2_visualizations

    _ = mcm2026c_q0_build_weekly_panel.run()
    _ = mcm2026c_q1_smc_fan_vote.run()

    q2_out = mcm2026c_q2_counterfactual_simulation.run()
    print('✅ Q2 done:', q2_out)

    q2 = pd.read_csv(q2_out.mechanism_comparison_csv)
    print('rows:', len(q2))
    display(q2.head(8))

    viz_cfg = VisualizationConfig()
    out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q2'])
    generate_all_q2_visualizations(REPO_ROOT, out_dirs['Q2'], viz_cfg, showcase=False)

    # Show TIFFs (best-effort)
    try:
        from IPython.display import Image, display as _display

        tiff_dir = REPO_ROOT / 'outputs' / 'figures' / 'q2' / 'tiff'
        candidates = [
            tiff_dir / 'q2_mechanism_difference_distribution.tiff',
            tiff_dir / 'q2_judge_save_impact.tiff',
            tiff_dir / 'q2_controversial_seasons_heatmap.tiff',
        ]

        shown = 0
        for fp in candidates:
            if fp.exists():
                _display(Image(filename=str(fp)))
                shown += 1
        if shown == 0:
            print('⚠️ No TIFF figures found to display yet. They should be under:', tiff_dir)
    except Exception:
        pass

except Exception as e:
    print('❌ One-click Q2 failed:', e)
    traceback.print_exc()

## 1) 确保 Q1 已运行（必要依赖）

如果你已经跑过 `run_all.py`，可以跳过。否则这里会先跑 Q0→Q1。

In [ ]:
from mcm2026.pipelines import mcm2026c_q0_build_weekly_panel, mcm2026c_q1_smc_fan_vote

_ = mcm2026c_q0_build_weekly_panel.run()
_ = mcm2026c_q1_smc_fan_vote.run()
'ok'


## 2) 运行 Q2

Q2 会聚合出每季层面的：
- Percent vs Rank 的差异周数
- Judge Save 的影响
- 与观察到的历史淘汰的一致性（match_rate）


In [ ]:
from mcm2026.pipelines import mcm2026c_q2_counterfactual_simulation

q2_out = mcm2026c_q2_counterfactual_simulation.run()
q2_out


In [ ]:
q2 = pd.read_csv(q2_out.mechanism_comparison_csv)
q2.head(8)


## 3) 快速读表：哪些季度对规则最“敏感”？

下面我们做一个论文友好的 summarization：
- 先把 `diff_weeks_percent_vs_rank` 最大的季度找出来（这些季度说明“规则改变会显著改变淘汰路径”）
- 再画一个简单柱状图，直观看 top seasons

这段结果通常可以直接写进论文的“机制差异实证”段落。

## 常见问答（FAQ）

### Q2 结果到底想说明什么？
一句话：
> 规则不是“写在纸上好看”，它真的会改变历史淘汰路径。

如果 `diff_weeks_percent_vs_rank` 很大，说明这个季度对规则很敏感。

### `match_rate_*` 是不是越高越好？
一般来说越高说明规则更贴近历史，但要注意：
- 历史规则本身未必“公平”
- 我们做 match_rate 更多是为了对模型/机制做 sanity check：
  至少不能离谱到完全解释不了历史。

### 为什么 Q2 需要 Q1？
因为 Q2 需要一个“观众投票”的代理。
Q1 给了我们每周的 `fan_share` / `fan_rank` 估计，Q2 才能在不同规则下重放淘汰。

### 为什么要看 Judge Save？
Judge Save 可以理解成“评委在关键时刻有否决权”。
它通常会：
- 提升技术保护（更难让技术差的人靠人气一路走）
- 但也可能降低粉丝表达（观众觉得‘投票没用’）

所以它是很适合在论文里讨论的一种制度变体。

In [ ]:
import matplotlib.pyplot as plt

# 找出“规则最敏感”的季度：diff_weeks_percent_vs_rank 最大
if 'diff_weeks_percent_vs_rank' in q2.columns:
    top = q2.sort_values('diff_weeks_percent_vs_rank', ascending=False).head(10).copy()
else:
    top = q2.head(10).copy()

cols = [c for c in ['season', 'diff_weeks_percent_vs_rank', 'match_rate_percent', 'match_rate_rank'] if c in top.columns]
print('Top seasons by divergence:')
top[cols]

if 'diff_weeks_percent_vs_rank' in top.columns:
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.bar(top['season'].astype(int).astype(str), top['diff_weeks_percent_vs_rank'].astype(float), color='steelblue', alpha=0.85)
    ax.set_title('Which seasons are most sensitive to rule change? (Percent vs Rank)')
    ax.set_xlabel('Season')
    ax.set_ylabel('# divergent weeks')
    ax.grid(True, axis='y', alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 3) 生成 Q2 图

图的输出目录：`outputs/figures/Q2/`（TIFF + EPS）。

In [ ]:
from mcm2026.visualizations.config import VisualizationConfig, create_output_directories
from mcm2026.visualizations.q2_visualizations import generate_all_q2_visualizations

viz_cfg = VisualizationConfig()
out_dirs = create_output_directories(REPO_ROOT / 'outputs' / 'figures', ['Q2'])
generate_all_q2_visualizations(REPO_ROOT, out_dirs['Q2'], viz_cfg, showcase=False)
out_dirs['Q2']
